In [1]:
'''
Importing all the requierd libraries 

'''
import numpy as np
import time
import os
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta, date
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import iplot

In [2]:
''' 
Nifty 50 stocks import 

'''
def get_50nsedata():
    data = pd.read_csv('ind_nifty50.csv')
    nse_50data = data['Symbol'].to_list()
    nse_50data = nse_50data[1:]
    return nse_50data

In [3]:
'''
Nifty 200 stocks import
'''
def get_200nsedata():
    data = pd.read_csv('ind_nifty200.csv')
    nse_200data = data['Symbol'].to_list()
    return nse_200data

In [4]:
'''
Download data from Yahoo finance based on stock name and date, interval
'''
def get_histdata(ticker, startdate, enddate, interval,Nse_data):
    tickk = ["{}.ns".format(tick) for tick in ticker]
    for line in tickk:
        symbol = line.split(",")[0]
        data = yf.download(symbol, start=startdate, end=enddate, interval=interval)
        data = data[(data[['Volume']] != 0).all(axis=1)]
        if Nse_data == 'Nifty50':
            data.to_csv('C:/Users/admin/Desktop/option chain analysis/datasets/daily/{}.csv'.format(symbol))
        elif Nse_data == 'Nifty200':
            data.to_csv('C:/Users/admin/Desktop/option chain analysis/datasets/monthly/{}.csv'.format(symbol))
        elif Nse_data == '50stocks':
            data.to_csv('C:/Users/admin/Desktop/option chain analysis/datasets/Nifty50/{}.csv'.format(symbol))
        elif Nse_data == '200stocks':
            data.to_csv('C:/Users/admin/Desktop/option chain analysis/datasets/Nifty200/{}.csv'.format(symbol))
        else:
            pass
    return data

In [5]:
def Support_resistance(Nse_data):
    if Nse_data == 'Nifty50':
        for stockname in os.listdir('datasets/daily'):
            rate = pd.read_csv('datasets/daily/{}'.format(stockname))
            if rate['Date'].iloc[-1] == date.today():
                ind = -1
            else:
                ind = -2
            Symbol = stockname.replace('.csv','')
            bala = pivot_cal(rate)
            bala.to_csv('C:/Users/admin/Desktop/option chain analysis/datasets/daily/{}.csv'.format(Symbol))
            
    elif Nse_data == 'Nifty200':
        for stockname in os.listdir('datasets/monthly'):
            rate = pd.read_csv('datasets/monthly/{}'.format(stockname))
            if rate['Date'].iloc[-1] == date.today():
                ind = -1
            else:
                ind = -2
            Symbol = stockname.replace('.csv','')
            bala = pivot_cal(rate)
            bala.to_csv('C:/Users/admin/Desktop/option chain analysis/datasets/monthly/{}.csv'.format(Symbol))
    else:
        pass
    return ind

In [6]:
def pivot_cal(rate):
    pivot              = round((rate['High']+rate['Low']+rate['Close'])/3 ,2)
    rate['Pivot']      = pivot
    support_1          = round((pivot*2)-rate['High'],2)
    rate['support_1']  = support_1
    resistance_1       = round((pivot*2)-rate['Low'],2)
    rate['resistance_1'] = resistance_1
    support_2          = round(pivot -(rate['High']-rate['Low']),2)
    rate['support_2']  = support_2
    resistance_2       = round(pivot +(rate['High']-rate['Low']),2)
    rate['resistance_2'] = resistance_2
    support_3          = round(rate['Low']-2*(rate['High']-pivot),2)
    rate['support_3']  = support_3
    resistance_3       = round(rate['High']+2*(pivot-rate['Low']),2)
    rate['resistance_3'] = resistance_3
    return rate

In [7]:
def Stock_inrange(Nse_data, pos):
    support_pivot = []
    pivot_r1 = []
    resistance1_up = []
    accu_out1 = []
    accu_out2 = []
    accu_out3 = []
    if Nse_data == 'Nifty50':
        for supres in os.listdir('datasets/daily'):   #support resistnce 
            support = pd.read_csv('datasets/daily/{}'.format(supres))
            for tmframe in os.listdir('datasets/Nifty50'):#daily stock data
                dapri = pd.read_csv('datasets/Nifty50/{}'.format(tmframe))
                if supres == tmframe:
                    cur_price = float(round(dapri['Close'].iloc[-1],2))
                    sup_zone = float(round(support['support_1'].iloc[pos],2))
                    pivot_zone = float(round(support['Pivot'].iloc[pos],2))
                    res_zone = float(round(support['resistance_1'].iloc[pos],2))
                    if ((cur_price >= sup_zone) and (cur_price <= pivot_zone)):
                        support_pivot.append(tmframe.replace('.ns.csv',''))
                        accu_out = more_accuracy(dapri,tmframe)
                        accu_out1.append(accu_out)
                    elif ((cur_price >= pivot_zone) and (cur_price <= res_zone)):
                        pivot_r1.append(tmframe.replace('.ns.csv',''))
                        accu_out = more_accuracy(dapri,tmframe)
                        accu_out2.append(accu_out)
                    elif cur_price > res_zone:
                        resistance1_up.append(tmframe.replace('.ns.csv',''))
                        accu_out = more_accuracy(dapri,tmframe)
                        accu_out3.append(accu_out)
                    else:
                        pass
    elif Nse_data == 'Nifty200':
         for supres in os.listdir('datasets/monthly'):   #support resistnce 
            support = pd.read_csv('datasets/monthly/{}'.format(supres))
            for tmframe in os.listdir('datasets/Nifty200'):#daily stock data
                dapri = pd.read_csv('datasets/Nifty200/{}'.format(tmframe))
                if supres == tmframe:
                    cur_price = float(round(dapri['Close'].iloc[-1],2))
                    sup_zone = float(round(support['support_1'].iloc[pos],2))
                    pivot_zone = float(round(support['Pivot'].iloc[pos],2))
                    res_zone = float(round(support['resistance_1'].iloc[pos],2))
                    if ((cur_price >= sup_zone) and (cur_price <= pivot_zone)):
                        support_pivot.append(tmframe.replace('.ns.csv',''))
                        accu_out = more_accuracy(dapri,tmframe)
                        accu_out1.append(accu_out)
                    elif ((cur_price >= pivot_zone) and (cur_price <= res_zone)):
                        pivot_r1.append(tmframe.replace('.ns.csv',''))
                        accu_out = more_accuracy(dapri,tmframe)
                        accu_out2.append(accu_out)
                    elif cur_price > res_zone:
                        resistance1_up.append(tmframe.replace('.ns.csv',''))
                        accu_out = more_accuracy(dapri,tmframe)
                        accu_out3.append(accu_out)
                    else:
                        pass
    else:
        pass
    return support_pivot,pivot_r1,resistance1_up,accu_out1,accu_out2,accu_out3
'''
    total = support_pivot+pivot_r1+resistance1_up
    accu_out1 = more_accuracy(total,tmframe)
    print(accu_out1)
'''

'\n    total = support_pivot+pivot_r1+resistance1_up\n    accu_out1 = more_accuracy(total,tmframe)\n    print(accu_out1)\n'

In [8]:
def more_accuracy(rate,stockname):
    if round(rate['Close'].iloc[-1].item(),2) > round(rate['Open'].iloc[-1].item(),2) and round(rate['Close'].iloc[-2].item(),2) > round(rate['Open'].iloc[-2].item(),2) and round(rate['Close'].iloc[-3].item(),2) > round(rate['Open'].iloc[-3].item(),2):
        return stockname.replace('.ns.csv','')
    else:
        pass

In [9]:
def candlestick(support,dapri,tmframe):
    trace1 = go.Candlestick(x=dapri['Datetime'],
                open=dapri['Open'], high=dapri['High'],
                low=dapri['Low'], close=dapri['Close'])
                     
    trace2 = go.Scatter(x=support['Date'],y=support['Pivot'],name = 'Pivot point')
    trace3 = go.Scatter(x=support['Date'],y=support['support_1'],name = 'S1')
    #trace4 = go.Scatter(x=support['Date'],y=support['support_2'],name = 'S2')
    #trace5 = go.Scatter(x=support['Date'],y=support['support_3'],name = 'S3')
    trace6 = go.Scatter(x=support['Date'],y=support['resistance_1'],name = 'R1')
    #trace7 = go.Scatter(x=support['Date'],y=support['resistance_2'],name = 'R2', )
    #trace8 = go.Scatter(x=support['Date'],y=support['resistance_3'],name = 'R3', )
    
    #data = (trace1,trace2,trace3,trace4,trace5,trace6,trace7,trace8)
    data = (trace1,trace2,trace3,trace6)
    layout = dict(title = tmframe)
    
    fig = dict(data=data, layout=layout)
    iplot(fig)
    

In [10]:
if __name__ == '__main__':
    #get the input from user nifty50 or nifty200
    Nse_data = str(input('Enter Nifty50 or Nifty200 : '))
    if Nse_data == 'Nifty50':
        stocky = '50stocks'
        Nifty_data = get_50nsedata()
    elif Nse_data == 'Nifty200':
        stocky = '200stocks'
        Nifty_data = get_200nsedata()
    else:
        pass
    
    # Pivot point calculation (support and resistance zone)
    interval = str(input('Enter the Time interval like: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk : '))
    if interval in ['1m', '2m', '5m', '15m']:  #last 60 days (1 month)
        inter = '1d'
        startdate = datetime.today() - timedelta(60)
    elif interval in ['30m', '60m', '90m', '1h']:    #last 200 days (5 months)
        inter = '1wk'
        startdate = datetime.today() - timedelta(200)
    elif interval in ['1d', '5d', '1wk']:          #last 200 days (5 months)
        inter = '1m'
        startdate = datetime.today() - timedelta(200)
    else:
        pass
    enddate = datetime.today()
    Hst_data = get_histdata(Nifty_data, startdate, enddate, inter,Nse_data)
    pos = Support_resistance(Nse_data)
    #percent = float(input('enter the percentage : '))
    
    #routine flow
    
    i=1
    while i<=1:
        startdate = date.today() - timedelta(30)
        enddate = datetime.today()
        print(stocky)
        time_data = get_histdata(Nifty_data, startdate, enddate, interval,stocky)
        print('the pos is : ', Nse_data)
        sup1, sup2, sup3, s_and_p, p_and_r, r_and_m = Stock_inrange(Nse_data, pos)
        s_and_p = list(filter(lambda item: item is not None, s_and_p))
        p_and_r = list(filter(lambda item: item is not None, p_and_r))
        r_and_m = list(filter(lambda item: item is not None, r_and_m))
        print('support to pivot zone : ', sup1)
        print('pivot to resistance zone : ', sup2)
        print('greater than resistance zone : ', sup3)
        print('last 3 green candle, support to pivot zone : ', s_and_p)
        print('last 3 green candle, pivot to resistance zone : ', p_and_r)
        print('last 3 green candle, resistance zone : ', r_and_m)
        print('completed')
        i = i+1


Enter Nifty50 or Nifty200 : Nifty200
Enter the Time interval like: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk : 5m
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********